In [ ]:
About the Dataset:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks whether the news article is real or fake:
    1: Fake news
    0: real News

In [3]:
import numpy as np
import pandas as pd
import re  # regular expression library,useful searching a paragraph or text
from nltk.corpus import stopwords #stopwords is the words that doesnt add values to the para
from nltk.stem.porter import PorterStemmer #used to stem our words, gives the root words of words
from sklearn.feature_extraction.text import TfidfVectorizer #used to convert the text into feature vectors
from sklearn.model_selection import train_test_split #splitting 
from sklearn.linear_model import LogisticRegression #model
from sklearn.metrics import accuracy_score #accuracy check

In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MRIDUL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
#Data preprocessing#
#loading the dataset to a pandas Dataframe
news_data = pd.read_csv("train.csv")

In [7]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news_data.shape

(20800, 5)

In [9]:
#findiing the missing value
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
#if the dataset have not enough data then we should not drop the values,
#but in this case there are enough data so we can drop or nullify
#replacing the null values with empty string
news_data = news_data.fillna(' ')

In [12]:
#we will use title and aurthor to predict, we will be leaving text because the it is very large,
#marging the aurthor name and news title
news_data['content'] = news_data['author']+' '+news_data['title']
news_data.head(2)

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."


In [13]:
#seperating the data and label
x = news_data.drop('label', axis=1)
y = news_data['label']
print(x)
print(y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

In [14]:
#Stemming
#Stemming is the process of reducing a word to its root word, it will remove all the prefix and suffix
#ex: actor, actress, acting ====== root is act
porter_stem = PorterStemmer()

In [21]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-z]',' ', content) #in the dataset, there are maybe numbers or symbols , we want to delete all the numbers and sybols fromthe content column
    stemmed_content = stemmed_content.lower() # converting all the words to lowercase letter
    stemmed_content = stemmed_content.split() #all the words will be converted into list
    stemmed_content = [porter_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #stemming , we are removing all the stopwords and choosing only the not stopwards
    stemmed_content = ' '.join(stemmed_content) #joining all the words
    return stemmed_content


In [23]:
news_data['content'] = news_data['content'].apply(stemming) #stemming the content column

In [24]:
print(news_data['content'])

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [25]:
#seperatinmg the data and label
x = news_data['content'].values
y = news_data['label'].values
print(x)
print(y)

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [26]:
#coverting all the texts to numerical values
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [27]:
print(x)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210529 stored elements and shape (20800, 16999)>
  Coords	Values
  (0, 264)	0.2701012497770876
  (0, 2466)	0.36765196867972083
  (0, 2936)	0.24684501285337127
  (0, 3570)	0.3598939188262558
  (0, 3762)	0.27053324808454915
  (0, 4924)	0.23331696690935097
  (0, 6938)	0.2187416908935914
  (0, 7619)	0.24785219520671598
  (0, 8553)	0.2921251408704368
  (0, 8829)	0.36359638063260746
  (0, 13357)	0.2565896679337956
  (0, 15566)	0.2848506356272864
  (1, 1482)	0.2957471154505952
  (1, 1878)	0.15614790568229528
  (1, 2207)	0.36915639258038363
  (1, 2793)	0.19208753385709676
  (1, 3538)	0.2653147533915268
  (1, 5448)	0.7186013955384664
  (1, 6752)	0.19152496072048605
  (1, 16671)	0.3025156488372128
  (2, 2920)	0.3179886800654691
  (2, 3075)	0.46097489583229645
  (2, 5334)	0.3866530551182615
  (2, 5911)	0.3474613386728292
  (2, 9538)	0.49351492943649944
  :	:
  (20797, 3613)	0.211655450844435
  (20797, 6977)	0.21809398920480086
  (20797

In [29]:
#splitting the dataset to training and test data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, stratify=y, random_state=42)

In [30]:
#training the model Logistic Regression Model
model = LogisticRegression()

In [31]:
model.fit(x_train,y_train)

LogisticRegression()

In [32]:
#finding the accuracy score

In [33]:
#evaluation
#accuracy for training data
x_train_pre = model.predict(x_train)
training_data_accu = accuracy_score(x_train_pre, y_train)
print(training_data_accu)

0.9873197115384615


In [34]:
#accuracy on test data
x_test_pre = model.predict(x_test)
test_data_accu = accuracy_score(x_test_pre, y_test)
print(test_data_accu)

0.975


In [37]:
#building a preditictive system

x_news = x_test[0]
prediction = model.predict(x_news)
print(prediction)
if (prediction[0]==0):
        print("The News is real")
else:
        print("The news is fake")

[0]
The News is real
